In [1]:
import torch
import math

# Example 1

## y = x^2

In [2]:
def dy_dx(x):
  return 2*x

dy_dx(3)

6

In [3]:
x = torch.tensor(3.0, requires_grad=True)
y = x**2

In [4]:
print(x)
print(y)

tensor(3., requires_grad=True)
tensor(9., grad_fn=<PowBackward0>)


In [5]:
y.backward()

x.grad

tensor(6.)

# Example 2

## y = x^2, z = sin(y)

In [6]:
def dz_dx(x):
    return 2 * x * math.cos(x**2)

dz_dx(3)

-5.466781571308061

In [7]:
x = torch.tensor(3.0, requires_grad=True)
y = x ** 2
z = torch.sin(y)

In [8]:
print(x)
print(y)
print(z)

tensor(3., requires_grad=True)
tensor(9., grad_fn=<PowBackward0>)
tensor(0.4121, grad_fn=<SinBackward0>)


In [9]:
z.backward()

x.grad

tensor(-5.4668)

In [10]:
y.grad

/tmp/ipykernel_55/486760323.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /pytorch/build/aten/src/ATen/core/TensorBody.h:489.)
  y.grad


# Example 3

## Neural Network

### Manually

In [11]:
# Inputs
x = torch.tensor(6.7)  # Input feature
y = torch.tensor(0.0)  # True label (binary: 0 or 1)

w = torch.tensor(1.0)  # Weight
b = torch.tensor(0.0)  # Bias

In [12]:
# Binary Cross-Entropy Loss for scalar
def binary_cross_entropy_loss(prediction, target):
    epsilon = 1e-8  # To prevent log(0)
    prediction = torch.clamp(prediction, epsilon, 1 - epsilon)
    return -(target * torch.log(prediction) + (1 - target) * torch.log(1 - prediction))

In [13]:
# Forward pass
z = w * x + b  # Weighted sum (linear part)
y_pred = torch.sigmoid(z)  # Predicted probability

# Compute binary cross-entropy loss
loss = binary_cross_entropy_loss(y_pred, y)

In [14]:
print(z)
print(y_pred)
print(loss)

tensor(6.7000)
tensor(0.9988)
tensor(6.7012)


In [15]:
# Derivatives:
# 1. dL/d(y_pred): Loss with respect to the prediction (y_pred)
dloss_dy_pred = (y_pred - y)/(y_pred*(1-y_pred))

# 2. dy_pred/dz: Prediction (y_pred) with respect to z (sigmoid derivative)
dy_pred_dz = y_pred * (1 - y_pred)

# 3. dz/dw and dz/db: z with respect to w and b
dz_dw = x  # dz/dw = x
dz_db = 1  # dz/db = 1 (bias contributes directly to z)

dL_dw = dloss_dy_pred * dy_pred_dz * dz_dw
dL_db = dloss_dy_pred * dy_pred_dz * dz_db

In [16]:
print(f"Manual Gradient of loss w.r.t weight (dw): {dL_dw}")
print(f"Manual Gradient of loss w.r.t bias (db): {dL_db}")

Manual Gradient of loss w.r.t weight (dw): 6.691762447357178
Manual Gradient of loss w.r.t bias (db): 0.998770534992218


### Use AutoGrad

In [17]:
x = torch.tensor(6.7)
y = torch.tensor(0.0)

In [18]:
w = torch.tensor(1.0, requires_grad=True)
b = torch.tensor(0.0, requires_grad=True)

In [19]:
print(w, b)

tensor(1., requires_grad=True) tensor(0., requires_grad=True)


In [20]:
z = w*x + b
y_pred = torch.sigmoid(z)
loss = binary_cross_entropy_loss(y_pred, y)

In [21]:
print(z)
print(y_pred)
print(loss)

tensor(6.7000, grad_fn=<AddBackward0>)
tensor(0.9988, grad_fn=<SigmoidBackward0>)
tensor(6.7012, grad_fn=<NegBackward0>)


In [22]:
loss.backward()

In [23]:
print(w.grad)
print(b.grad)

tensor(6.6918)
tensor(0.9988)


### Using Vector

In [24]:
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
x

tensor([1., 2., 3.], requires_grad=True)

In [25]:
y = (x**2).mean()
y

tensor(4.6667, grad_fn=<MeanBackward0>)

In [26]:
y.backward()

x.grad

tensor([0.6667, 1.3333, 2.0000])

# Clearing Grad

In [27]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [28]:
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

In [29]:
y.backward()

x.grad

tensor(4.)

In [30]:
x.grad.zero_()

tensor(0.)

# Disable Gradient Tracking

In [31]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [32]:
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

In [33]:
y.backward()

x.grad

tensor(4.)

Three techniques:
option 1 - requires_grad_(False)
option 2 - detach()
option 3 - torch.no_grad()

## Option 1: requires_grad_(False)

In [34]:
x.requires_grad_(False)
x

tensor(2.)

In [35]:
y = x ** 2
y

tensor(4.)

In [36]:
#y.backward()
# this give a error, because here not grad is false

## Option 2: detach()

In [37]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [38]:
z = x.detach()
z

tensor(2.)

In [39]:
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

In [40]:
y1 = z ** 2
y1

tensor(4.)

In [41]:
y.backward()

In [42]:
#y1.backward()
# here give the error because of detach()

## Option 3: torch.no_grad()

In [43]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [44]:
with torch.no_grad():
    y = x ** 2

y

tensor(4.)

In [45]:
#y.backward()
# here give the error because of torch.no_grad()